In [4]:
# importando bibliotecas
import pandas as pd
import sklearn as sk
import os

In [7]:
data = os.getcwd() + "/train_data.csv" # os.getcwd() gets the current directory $(pwd)
adult_brute = 

'/home/11224185/workspace/knn/train_data.csv'